# 存在しないデータの扱い
## 存在しないデータへの対応のトレードオフ
存在しないデータに対しては2つの戦略が一般的に取られる。値が存在しないことを示すマスクを利用するか、エントリーが存在しないを示す番兵としての値を選択するかである。

マスクのアプローチでは、マスクはBooleanのarrayに分けられるか、nullステータスの値をローカルに示す1ビットをデータの中に含めたりするだろう。

番兵のアプローチでは、番兵の値はデータ特有のルールになるであろう。それは存在しない整数値として-9999や特定のビットパターンであったり、もしくはより広いルールを用いて、NaNなどを利用するかもしれない。

これらのアプローチはトレードオフなしではない。分けられたマスクのarrayを使用することは余分なBoolean arrayの確保が必要となり、それはストレージと計sンにおいてオーバーヘッドを追加する。番兵の値は表現できる有効な値の範囲を減らし、CPUやCPUでの計算において余分なロジックを必要とするであろう。一般に特別な値であるNaNであっても全てのデータタイプで利用することは出来ない。

いずれの場合であっても全般的に最適の選択は無く、異なる言語や異なるシステムではことなるルールを使用する。例えば、R言語は存在しないデータを示すのにデータタイプごとの番兵の値のビットパターンを予約しているし、一方SciDBシステムはNA状態を示すのにセルごとに割り付けた余分なバイトを利用している。

## Pandasにおける存在しないデータの扱い
なんやかんやあって、PandasではPythonですでに存在するnull値である`NaN`と`None`オブジェクトを選択する。

### Pythonicなmissing data: `None`
`None`はPythonオブジェクトであるため、任意のどんなNumPyやPandasのarrayでも使用することは出来ないが、`object`タイプのarrayでのみ利用することが出来る。

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

`dtype=object`はいろんなデータを表現できる一方で、オーバーヘッドも大きい。

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
106 ms ± 7.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.38 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



array全域にわたる`sum()`や`min()`を`None`を含んでいるarrayに対して行うとエラーとなる。

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### 存在しない数値データ： `NaN`
`NaN`は標準的なIEEE浮動小数点表現を利用する全てのシステムによって認識される特別な浮動小数点の値である。

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

この場合、`NaN`を含んでいるが、ネイティブな浮動小数点型であるため、このarrayはコンパイルコードに収められている早い計算をサポートしている。

どんな計算に関わらず、`NaN`を含む計算結果は`NaN`となる。

In [6]:
1 + np.nan

nan

In [7]:
0 * np.nan

nan

つまり、`sum()`などにエラーは出ないが、その結果自体には意味がない。

In [8]:
vals2.sum(), vals2.min(), vals2.max()

/home/sasaki/.pyenv/versions/miniconda3-4.3.30/envs/ml_env/lib/python3.7/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/home/sasaki/.pyenv/versions/miniconda3-4.3.30/envs/ml_env/lib/python3.7/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


(nan, nan, nan)

NumPyは`NaN`を無視する特別なアグリゲーションを用意している。

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

`NaN`は特別な浮動小数点の値であるため、それと同等な値はintegersやstrings、そのほかの型にはない。

### PandasにおけるNaNとNone
Pandasでは`NaN`と`None`はほぼ互いに変換可能となっており、それらは適切に変換される。

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

NA値が存在する時、Pandasでは利用可能な番兵の値が無い型に対して、自動的に型のキャストを行う。

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

## Null値に対する計算
PandasではNull値に対する検出や削除、置換を行う便利なメソッドがいくつかある。

- `isnull()`: Generate a boolean mask indicating missing values
- `notnull()`: Opposite of isnull()
- `dropna()`: Return a filtered version of the data
- `fillna()`: Return a copy of the data with missing values filled or imputed

### Null値の検出

In [13]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Null値の切り落とし

In [16]:
data.dropna()

0        1
2    hello
dtype: object

`DataFrame`に対しては、いくつかのオプションがある。

In [17]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


`DataFrame`からは単独の値をドロップすることは出来ない。出来るのは行か列全体をドロップするのみである。

デフォルトでは`dropna()`はnull値が存在する全ての行をドロップする。

In [18]:
df.dropna()

,0,1,2
1,2.0,3.0,5


代わりに、`axis`キーワードを利用すれば、異なる軸でドロップすることも出来る。

In [19]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


しかし、このドロップはいくつかの良いデータまでもドロップしてしまう。全ての値がNA、もしくはほとんどがNAの行や列のみを削除したいかもしれない。これらは`how`や`thresh`を指定して実行することができる。

デフォルトで`how='any'`となっていて、一つでもnull値があると削除されてしまうが、`how='all'`を指定すれば、すべてがnull値である行や列のみを削除できる。

In [20]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [21]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


`thresh`パラメーターはnull値でない値が最小いくつ含まれていなければならないか指定して、行や列を削除できる。

In [22]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### null値を満たす

In [23]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [24]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [25]:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [26]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [27]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [28]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
